## Практическое задание к уроку № 3 по теме "Коллаборативная фильтрация."

## **Задание 1.**

# Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его существенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
### А точно нужно сортировать по вероятности?  
Сортировать нужно по вероятности, но учитывая разделение по группам.  
Например, предлагать разные товары либо по категориям товаров, либо по  
цене, или ещё как-то. При этом из каждой группы будем брать товар с наибольшей  
вероятностью.
### Какую метрику использовать?  
Для оценки модели можно использовать три метрики вместе:  
1: Precision@k для оценки качества рекомендаций,  
насколько они актуальны для пользователей;  
2: MAP@k для оценки правильности порядка, в котором  
рекомендации выдаются;  
3: Money precision@k для расчёта финансового эффекта  
от рекомендаций.  
### Сколько раз в неделю отправляем рассылку?  
Пуш-уведомление с одной самой топовой рекомендацией можно отправлять  
каждый день. Например, новая скидка на товар, который пользователь покупает.  
Либо новое поступление похожего товара.  
На электронную почту будем отправлять раз в неделю. Там будет больше  
рекомендаций, ну и будем стараться не надоедать спамом.
### В какое время отправляем рассылку?  
Пуш-уведомление лучше отправлять утром, если у нас нет данных по конкретным  
клиентам: работает он или пенсионер. Утром ещё не успеют сходить в магазин те,  
кто не работает. Кто на работе, тот может построить план зайти в магазин после  
работы за нашим товаром. Также может позвонить своим неработающим родственникам,  
чтобы они зашли и купили. Лучшее время - 10:00.  
На электронную почту лучше отправлять вечером в четверг. Тогда люди к пятнице  
уже должны будут увидеть письмо и скорректировать свои покупки в выходные.  
### Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?  
Можно ставить фильтр на те товары, которые пользователь уже купил. Также,  
если человек не покупает товары из наших рекомендаций, то модель постепенно  
подстроится и перестанет их рекомендовать. Веса в таблице user_item будут  
меняться в зависимости от актуальных скидок. Если применяем взвешивание, то  
веса будут меняться в зависимости от покупок других людей. Если используем  
модель ALS, то можно периодически её переобучать с разным random_state.  
### Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?  
Да, чтобы не было взаимоисключающих рекомендаций. Например, две разных марки  
молока - пользователь всё равно выберет что-то одно. Либо один и тот же товар,  
но в пачках разного объёма - нет смысла предлагать две одинаковых сметаны,  
одна 200гр., другая - 500гр. Для этого нужна разбивка товаров по категориям.  
Далее можно настроить фильтрацию при выдаче рекомендаций, чтобы для каждой  
категории товаров выдавалось не более одной рекомендации. Определить различие  
товаров можно, например, посчитав их взаимное косинусное сходство. Также можно  
воспользоваться методом similar_items модели ALS, и взять оттуда непохожие друг на друга.  

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
### 3 товара с акцией(Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)  
Нужно повышать вес в user-item матрице. Например, при скидке 10% все веса для этого товара  
перед обучением модели умножаются на 1.1. При скидке 50% - на 1.5. Для удобства, в исходную   
матрицу нужно добавить столбик с величиной скидки для каждого товара (1 - нет скидки), а  
агрегатная функция, по которой мы будем создавать user-item матрицу, будет умножать вес на  
эту скидку для всех товаров. Также нужно будет создать категорию для акционных товаров  
дополнительно к обычной категории и фильтровать рекомендации модели на выдачу 3 акционных  
товаров.  
### 1 новый товар (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)  
На мой взгляд, идея фильтра на уже покупавшиеся товары хорошая. Если модель  
сделала такую рекомендацию, значит товар более-менее интересен пользователю.  
Низкая вероятность покупки может быть связана только со стоимостью. В этом  
случае можно не предлагать дорогие товары.  
### 1 товар для роста среднего чека (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)  
Одна из рекомендаций должна заменяться на похожий товар из этой же категории, но  
имеющий более высокую цену. Например, на 10% более высокую. Если такого товара  
нет, то взять другую рекомендацию и посмотреть по ней.

## **Задание 2.**

Поэкспериментировать с ALS (grid-search)  
Подобрать лучшие параметры  
Написать отчет об увиденном  

### 2.1 Исходные данные

Скопируем код с урока:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('../../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
result = data_test.groupby('user_id', sort=False)['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,790,"[822989, 849003, 884731, 920109, 976998, 98236..."
1,2252,"[9194207, 6534178, 916767, 947858, 1017061, 10..."


In [4]:
popularity = data_train.groupby('item_id', sort=False)['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [5]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0,
                                  sort=False
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
2375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Напишем функцию для обучения модели, зафиксируем random_state:

In [7]:
def train_als(matrix_csr, factors=64, regularization=0.05, iterations=15):

    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=regularization,
                                    iterations=iterations, 
                                    calculate_training_loss=True,
                                    random_state=29)

    model.fit(matrix_csr.T.tocsr(),  # На вход item-user matrix
              show_progress=False)
    
    return model

Обернём код в удобные для использования функции:

In [8]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999_999]], 
                                    recalculate_user=True)]
    return res

In [9]:
def evaluate_model(result_df, model, N=5):
    
    result_df['als'] = result_df['user_id'].apply(lambda x: get_recommendations(x, model=model, N=N))
    
    return result_df.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Сначала опробуем дефолтную модель на матрице из урока:

In [10]:
default_result = evaluate_model(result, train_als(csr_matrix(user_item_matrix)))
default_result

0.1749874434957308

### 2.2 Другое формирование матрицы user-item

Посмотрим, сколько в датасете строк с quantity == 0, которые нашей  
агрегатной функцией учитываются, как покупка:

In [11]:
(data_train['quantity'] == 0).sum()

13813

Сформируем матрицу user-item без учёта строк, где quantity = 0:

In [12]:
user_item_matrix = pd.pivot_table(data_train.loc[data_train['quantity'] != 0], 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0,
                                  sort=False
                                 )

user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix)

Переопределим словари маппинга:

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Оценим эффект:

In [14]:
new_matrix_result = evaluate_model(result, train_als(csr_matrix(user_item_matrix)))
new_matrix_result

0.1753892516323456

Модель получилась чуть точнее, далее будем работать с  
этой матрицей.

### 2.3 Взвешивание матрицы user-item

#### 2.3.1 Tfidf

In [15]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T

In [16]:
tfidf_result = evaluate_model(result, train_als(csr_matrix(user_item_matrix)))
tfidf_result

0.18372677046710198

#### 2.3.2 BM25

In [17]:
user_item_matrix = pd.pivot_table(data_train.loc[data_train['quantity'] != 0], 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0,
                                  sort=False
                                 )

user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix)

In [18]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [19]:
bm25_result = evaluate_model(result, train_als(csr_matrix(user_item_matrix)))
bm25_result

0.20050226017076847

BM25-взвешивание показало лучший результат, возьмём  
эту матрицу за основу и подберём параметры к модели.

### 2.4 Подбор параметров

In [20]:
from itertools import product

In [21]:
def find_best_params(params, verbose=False):
    
    parameters = list(params.keys())
    
    # Воспользуемся itertools для создания всевозможных
    # комбинаций параметров
    values = list(product(*params.values()))
    
    best_score = 0

    for params_set in values:
        
        # Превращаем набор параметров в словарь с названиями
        # и значениями
        current_params = {param[0]: param[1] for param in zip(parameters, params_set)}
        
        score = evaluate_model(result, train_als(csr_matrix(user_item_matrix), **current_params))
        
        if verbose:
            print(f'{current_params}\t\t{score:.3f}')
        
        # Лучшую метрику и параметры сохраняем
        if score > best_score:
            best_score = score
            best_params = current_params
                
    return best_score, best_params

In [22]:
params = {
    'factors': [32, 64, 128],
    'regularization': [0.01, 0.05, 0.1],
    'iterations': [10, 15, 20]
}

In [23]:
%%time

best_score, best_params = find_best_params(params, verbose=True)

{'factors': 32, 'regularization': 0.01, 'iterations': 10}		0.185
{'factors': 32, 'regularization': 0.01, 'iterations': 15}		0.188
{'factors': 32, 'regularization': 0.01, 'iterations': 20}		0.188
{'factors': 32, 'regularization': 0.05, 'iterations': 10}		0.188
{'factors': 32, 'regularization': 0.05, 'iterations': 15}		0.187
{'factors': 32, 'regularization': 0.05, 'iterations': 20}		0.187
{'factors': 32, 'regularization': 0.1, 'iterations': 10}		0.187
{'factors': 32, 'regularization': 0.1, 'iterations': 15}		0.186
{'factors': 32, 'regularization': 0.1, 'iterations': 20}		0.187
{'factors': 64, 'regularization': 0.01, 'iterations': 10}		0.198
{'factors': 64, 'regularization': 0.01, 'iterations': 15}		0.200
{'factors': 64, 'regularization': 0.01, 'iterations': 20}		0.200
{'factors': 64, 'regularization': 0.05, 'iterations': 10}		0.200
{'factors': 64, 'regularization': 0.05, 'iterations': 15}		0.201
{'factors': 64, 'regularization': 0.05, 'iterations': 20}		0.201
{'factors': 64, 'regularizat

Лучшие параметры получились:

In [24]:
best_params

{'factors': 128, 'regularization': 0.1, 'iterations': 20}

Лучшая метрика:

In [25]:
best_score

0.2237066800602712

Так как лучшими оказались пограничные значения параметров, то  
для более качественного подбора нужно ещё раз запустить поиск,  
с другими границами параметров. Но это не является нашей целью  
в этой домашней работе. Общие тенденции и так видны.

<ins>Вывод:</ins>  
На повышение точности рекомендаций сильно повлияло добавление  
взвешивания. Идея исключить покупки с количеством купленного  
товара, равного нулю, дала небольшой эффект, но это, скорее  
всего, ввиду того, что в исходном датасете было мало таких  
строк. Применение других агрегатных функций при создании user-item  
матрицы значительно повлияло бы на точность модели, но применённая  
на уроке функция, учитывающия количество фактов покупки уже достаточно  
неплохая, в этом мы убедились в предыдущей домашней работе, где  
сравнили этот подход с агрегацией суммы покупок.  
Подбор параметров показал, что наибольшее влияние на метрику оказывает  
количество факторов, тогда как значение регуляризации и количество  
итераций не оказывали сильного влияния.